In [1]:
import pandas as pd
import sqlite3
import glob
import os
import os.path as op

In [2]:
csv_path = op.join(os.getcwd(), '..', '..', 'data', 'fake_sepsis_data')
conn = sqlite3.connect("../../data/sepsis.db")

for file in glob.glob(op.join(csv_path, "*.csv")):
    print('Reading', file)
    df = pd.read_csv(file)
    # First column should be unnamed... if not, there's a problem
    if (df.columns[0] == 'Unnamed: 0'):
        df.drop('Unnamed: 0', axis=1, inplace=True)
    else:
        print('   --- Check Data File')
    filename, fileextension = op.splitext(file)
    filename = op.basename(filename)
    df.to_sql(filename, conn, if_exists='replace', index=False)
    conn.commit()
    print('Saved', filename, 'to database')

conn.close()


Reading /Users/seth/OneDrive - The University of Colorado Denver/Documents/ICML_2018_Paper/faker-prototype/sourcecode/python/../../data/fake_sepsis_data/admission.csv
Saved admission to database
Reading /Users/seth/OneDrive - The University of Colorado Denver/Documents/ICML_2018_Paper/faker-prototype/sourcecode/python/../../data/fake_sepsis_data/admission_diagnoses.csv
Saved admission_diagnoses to database
Reading /Users/seth/OneDrive - The University of Colorado Denver/Documents/ICML_2018_Paper/faker-prototype/sourcecode/python/../../data/fake_sepsis_data/adt_events.csv
Saved adt_events to database
Reading /Users/seth/OneDrive - The University of Colorado Denver/Documents/ICML_2018_Paper/faker-prototype/sourcecode/python/../../data/fake_sepsis_data/diagnoses.csv
Saved diagnoses to database
Reading /Users/seth/OneDrive - The University of Colorado Denver/Documents/ICML_2018_Paper/faker-prototype/sourcecode/python/../../data/fake_sepsis_data/flowsheet.csv
Saved flowsheet to database
Rea

In [3]:
con = sqlite3.connect("../../data/sepsis.db")

cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for c in tables:
    cursor.execute("SELECT sql FROM sqlite_master WHERE name='" + c[0] + "'")
    print(cursor.fetchall()[0][0])

con.close()

CREATE TABLE "admission" (
"SubjectId" INTEGER,
  "EncounterId" INTEGER,
  "DOB" TEXT,
  "Sex" TEXT,
  "Race" TEXT,
  "Ethnicity" TEXT,
  "FirstHeightInInches" INTEGER,
  "FirstWeightInOunces" INTEGER,
  "AdmittingDepartmentName" TEXT,
  "AdmissionDateTime" TEXT,
  "FinancialClass" TEXT,
  "AdmissionType" TEXT,
  "AdmissionSource" TEXT,
  "PatientClass" TEXT,
  "AdmittingService" TEXT,
  "PrincipalProblem" TEXT,
  "PrimaryCodedDx" TEXT,
  "DischargeDateTime" TEXT,
  "DischargeDisposition" TEXT,
  "HospitalService" TEXT
)
CREATE TABLE "admission_diagnoses" (
"SubjectId" INTEGER,
  "EncounterId" INTEGER,
  "AdmittingDiagnosis" TEXT
)
CREATE TABLE "adt_events" (
"SubjectId" INTEGER,
  "EncounterId" INTEGER,
  "DepartmentName" TEXT,
  "TransferInDateTime" TEXT,
  "TransferOutDateTime" TEXT
)
CREATE TABLE "diagnoses" (
"SubjectId" INTEGER,
  "EncounterId" INTEGER,
  "Source" TEXT,
  "StartDate" TEXT,
  "Code" TEXT,
  "Type" TEXT
)
CREATE TABLE "flowsheet" (
"SubjectId" INTEGER,
  "Encounter

In [13]:
import hug
from seth_fakelite3 import KungFauxPandas
import sqlite3
import sys


kfpd = KungFauxPandas()
dbname = 'file:../../data/sepsis.db?mode=ro'
db_conn = sqlite3.connect(dbname, uri=True)
cursor = db_conn.cursor()

df = kfpd.read_sql('select * from admission limit 10', db_conn, 'kde')
#df.style.hide_index()
df.to_html(classes='table')
df.style.hide_columns('index')

Processing column SubjectId as a int64
Processing column EncounterId as a int64
Processing column DOB as a object
Processing column Sex as a object
Processing column Race as a object
Processing column Ethnicity as a object
Processing column FirstHeightInInches as a int64
Processing column FirstWeightInOunces as a int64
Processing column AdmittingDepartmentName as a object
Processing column AdmissionDateTime as a object
Processing column FinancialClass as a object
Processing column AdmissionType as a object
Processing column AdmissionSource as a object
Processing column PatientClass as a object
Processing column AdmittingService as a object
Processing column PrincipalProblem as a object
Processing column PrimaryCodedDx as a object
Processing column DischargeDateTime as a object
Processing column DischargeDisposition as a object
Processing column HospitalService as a object


AttributeError: 'Styler' object has no attribute 'hide_columns'